# 🎯 Step 04: 데이터베이스 메타데이터 RAG 시스템

이 노트북에서는 실제 데이터베이스 메타데이터를 활용하여 실용적인 RAG 시스템을 구현합니다.
사용자는 자연어로 데이터베이스 구조에 대해 질문하고, 시스템은 메타데이터를 기반으로 답변합니다.

## 📚 1단계: 필요한 라이브러리 임포트

이전 단계의 라이브러리에 더해 다음 라이브러리들을 추가로 사용합니다:
- pandas: CSV 파일 읽기 및 데이터 처리
- json: 샘플 값과 통계 데이터 처리
- typing: 타입 힌트 제공

In [ ]:
# 기본 시스템 라이브러리
import os
import json
from typing import List, Dict
from dotenv import load_dotenv

# 데이터 처리 라이브러리
import pandas as pd

# Elasticsearch 관련
from elasticsearch import Elasticsearch

# LangChain 관련
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import ElasticsearchStore
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# 환경 변수 로드
load_dotenv()

## 📊 2단계: 메타데이터 로드 및 전처리

sample_metadata.csv 파일에는 다음과 같은 정보가 포함되어 있습니다:
- 테이블명
- 컬럼명
- 데이터 타입
- 설명
- 샘플 값
- 통계 정보

이 정보를 읽어서 RAG 시스템에서 사용할 수 있는 형태로 변환합니다.

In [ ]:
# CSV 파일 경로 설정
metadata_file = "../data/sample_metadata.csv"

# CSV 파일 읽기
df = pd.read_csv(metadata_file)

# 메타데이터를 문서화된 텍스트로 변환하는 함수
def format_metadata_to_text(row) -> str:
    # JSON 형식의 문자열을 파이썬 객체로 변환
    sample_values = json.loads(row['sample_values'].replace("'", '"'))
    statistics = json.loads(row['statistics'].replace("'", '"'))
    
    # 통계 정보를 읽기 쉬운 형식으로 변환
    stats_text = []
    for key, value in statistics.items():
        if key == 'null_count':
            stats_text.append(f"NULL 값 개수: {value}")
        elif key == 'distinct_count':
            stats_text.append(f"고유 값 개수: {value}")
        elif key in ['min', 'max']:
            stats_text.append(f"{key}: {value}")
    
    # 전체 텍스트 구성
    return f"테이블 '{row['table_name']}'의 '{row['column_name']}' 컬럼:\n" + \
           f"- 데이터 타입: {row['data_type']}\n" + \
           f"- 설명: {row['description']}\n" + \
           f"- 샘플 값: {', '.join(map(str, sample_values))}\n" + \
           f"- 통계: {', '.join(stats_text)}"

# 각 행을 문서화된 텍스트로 변환
documents = df.apply(format_metadata_to_text, axis=1).tolist()

# 변환된 문서 개수 출력
print(f"✅ {len(documents)}개의 메타데이터 문서가 생성되었습니다.")

# 첫 번째 문서 예시 출력
print("\n📝 문서 예시:")
print(documents[0])

## 🔌 3단계: Elasticsearch 연결

이전 단계와 동일하게 Elasticsearch에 연결합니다.
이번에는 메타데이터 전용 인덱스를 생성할 것입니다.

In [ ]:
# Elasticsearch 클라이언트 설정
es = Elasticsearch(
    os.getenv('ELASTICSEARCH_URL'),
    basic_auth=(
        os.getenv('ELASTICSEARCH_USERNAME'),
        os.getenv('ELASTICSEARCH_PASSWORD')
    ),
    verify_certs=False
)

# 연결 확인
if es.ping():
    print("✅ Elasticsearch 연결 성공!")
else:
    print("❌ Elasticsearch 연결 실패!")

## 🔄 4단계: 메타데이터 벡터 저장소 설정

메타데이터 문서들을 벡터로 변환하여 저장합니다.
이전과 달리 이번에는 실제 데이터베이스 구조에 대한 정보를 저장하므로,
더 정확한 검색과 답변이 가능해집니다.

In [ ]:
# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings()

# 메타데이터 전용 벡터 저장소 설정
index_name = "metadata-rag"  # 새로운 인덱스 이름
vector_store = ElasticsearchStore(
    es_connection=es,
    index_name=index_name,
    embedding=embeddings
)

# 메타데이터 문서들을 벡터로 변환하여 저장
vector_store.add_texts(documents)
print(f"✅ {len(documents)}개의 메타데이터 문서가 벡터 저장소에 저장되었습니다.")

## 🤖 5단계: 메타데이터 RAG 시스템 구현

이제 데이터베이스 메타데이터에 특화된 RAG 시스템을 구현합니다.
사용자는 다음과 같은 질문들을 할 수 있습니다:
- 특정 테이블의 구조
- 특정 컬럼의 데이터 타입
- 컬럼의 설명과 용도
- 데이터의 특성과 통계 정보

In [ ]:
# ChatGPT 모델 초기화
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# 메타데이터 질의응답 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

## 💬 6단계: 메타데이터 질의응답 테스트

이제 다양한 질문을 통해 메타데이터 RAG 시스템을 테스트해봅니다.
시스템은 질문의 의도를 파악하고, 관련된 메타데이터를 검색하여
정확하고 상세한 답변을 제공합니다.

In [ ]:
# 테스트할 질문들
questions = [
    "customer_orders 테이블의 구조를 설명해주세요.",
    "주문 금액의 범위는 어떻게 되나요?",
    "주문 상태는 어떤 값들이 가능한가요?",
    "고객의 이메일 정보는 어떤 테이블에 저장되나요?"
]

# 각 질문에 대한 답변 생성
for question in questions:
    print(f"\n질문: {question}")
    print(f"답변: {qa_chain.run(question)}")
    print("-" * 80)